In [1]:
import pandas as pd
import numpy as np
import re
from gensim.models import word2vec, Word2Vec
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy as np
from numpy import argmax
%matplotlib inline

from ckonlpy.tag import Twitter #konlpy twitter는 add_dictionary가 없어서, ckonlpy(customized konlpy) 사용.
twitter = Twitter()

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


## 입력데이터

In [2]:
df = pd.read_csv('final_data_19612.csv', sep='\t', encoding='UTF-8')
# df = df2.loc[df2['대상 선정']=='O',:]

In [3]:
df = df.fillna('N')

userdic = pd.read_csv('userdic_1906122.csv', sep='\t', encoding='UTF-8')
userdic['User Dic.'][0:1]
userdic = userdic.rename(columns = {'User Dic.':'standard'})
userdic = userdic.rename(columns = {'동의어':'missprint'})
userdic = userdic.rename(columns = {'불용단어':'stops'})
userdic_list = userdic.rename(columns = {'User Dic.':'standard'})
userdic_list = userdic.rename(columns = {'동의어':'missprint'})
userdic_list = userdic.rename(columns = {'불용단어':'stops'})
userdic_list = userdic_list.fillna('')
userdic = userdic.fillna('')
replace_word = userdic.iloc[:,1:3]
replace_word = replace_word[replace_word.missprint != '']

In [4]:
replaceword_list =  replace_word['standard'].tolist()
missword_list = replace_word['missprint'].tolist()
print(len(replaceword_list))
print(len(missword_list))

757
757


In [5]:
for i in range(len(replaceword_list)):
    df['clm_prtl_sbc'] = df['clm_prtl_sbc'].replace(missword_list[i],replaceword_list[i],regex=True)
#regex=True 설정을 주면 조사가 붙어있던 말던 단어를 다 바꿔준다.

In [42]:
df[df['clm_prtl_sbc'].isnull()]

Empty DataFrame
Columns: [std_clm_type_cd, ro_y, clm_jgmt_dt, ceq_caus_cd, ceq_psit_cd, ceq_psit_nm, trim_plnt_cd, prj_vehl_cd, mdl_cd, part_sys_cd, repn_part_cd, clm_prtl_sbc, 대상 선정, 구분, 책임구, 원인, target_pred]
Index: []

In [43]:
df[df['구분'].isnull()]

Empty DataFrame
Columns: [std_clm_type_cd, ro_y, clm_jgmt_dt, ceq_caus_cd, ceq_psit_cd, ceq_psit_nm, trim_plnt_cd, prj_vehl_cd, mdl_cd, part_sys_cd, repn_part_cd, clm_prtl_sbc, 대상 선정, 구분, 책임구, 원인, target_pred]
Index: []

## 텍스트 전처리

In [44]:
# 예측값과 실제값 비교를 위해 컬럼을 하나 더 생성합니다.
# df['result_pred'] = df['검토결과'].astype(str).copy()
# df['target_pred'] = df['대상 선정'].astype(str).copy()
# df['charge_pred'] = df['책임구'].astype(str).copy()
# df['division_pred'] = df['구분'].astype(str).copy()

### (1) 오타, 동의어 처리(replace)

### (2) 사용자 단어 정의(User dictionary)

for word in replaceword_list: 
    twitter.add_dictionary(word, 'Noun') # for문을 통해 dictionary에 단어들을 Noun으로 추가.

# twitter.pos(df['clm_prtl_sbc'][1]) # '리어', '미장착'이 Noun으로 잘 정의되었는지 확인.

### (3) 명사 추출(Twitter)

def nouns(text):
    text = str(text)
    text = twitter.nouns(text)
    return text

df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(nouns)

### (4) 특수문자, 띄어쓰기 전처리

def preprocessing(text):
    # 개행문자 제거
    text= str(text)
    text = re.sub('\\\\n', ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    #text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ]', ' ', text) 
    text = re.sub(' +', ' ', text)
    return text

# 불용어 제거
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ['(LH)','(RH)','(사진첨부)','(우)','(좌)','[C]','A/S','C','C-10000','C-10000','F/LH','F/RH','FR','FRT',
             'Frt','frt','L/H','L/H리어','LH','LH/RH','LH쪽','R/H','R/H리어','R/LH','R/RH','RH','RH쪽','Rr','RR','rr',
             '결과','고객','고객에게','공임','광주','그랜저','그랜져','낫','낮','넥쏘','니로','당','당차량','당차량은','대리점',
             '동영상참조','레이','렌트','리무진','리어LH','리어RH','모닝','모하비','밤','벨로스터','사진','사진참조','사진첨부',
             '상기','상기차량은','새벽','새차','서비스','소하','스타렉스','스팅어','스포티지','신차','신차','심야','싼타페',
             '써비스','쏘나타','쏘렌토','쏘울','아반떼','아산','아이오닉','아침','야간','언덕','엑센트','열','울산','원인',
             '임시 넘버','임시 번호','입고','자동차','저머검내용','점검/원인','점검결과','점검내용','점검및원인','점검및조치내용',
             '점검점검','점검점검내용','조치결과','조치내용','주간','주재원','주제원','주차장','증상','지하','진입','차량',
             '차량은','철도','첨부','첨부파일','청구','출고','카 마스터','카니발','카렌스','코나','클레임','투싼',
             '팰리세이드','하이리무진','하이브리드','현     상','현    상','현   상','현  상','현 상','현산','현상','현상및점검',
             '현상및조치','화성'
]
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words) #stops에 안 걸리는 나머지 것들 이어붙이기

%time df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(preprocessing)

%time df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(remove_stopwords)

df['clm_prtl_sbc'].head()

Wall time: 483 ms
Wall time: 642 ms


0     엔진 룸 다량 연료 누유 발생 점검 엔진 룸 다량 연료 누유 점검 점검 내용 엔진...
1                                리어 도어 안티 치핑 필름 장착 불량 
2                      커튼 홀더 장착 불량 조치 내용 커튼 홀더 장착 후 함 
3                  및 조치 시 웨더 스트림 스트림 무 주름 발생 들뜸 교환 조치 
4                             몰딩 내부 고무 조립 불량 상태 몰딩 조립 
Name: clm_prtl_sbc, dtype: object

In [47]:
df.head()

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd    ceq_psit_nm  \
0               W  2017     20170811         ZZ7           N  조립 문제_미장착/오장착   
1               W  2018     20180822         ZZ7         B11  조립 문제_미장착/오장착   
2               W  2018     20181006         ZZ5         B11  조립 문제_미장착/오장착   
3               W  2018     20180928         ZZ7         B11  조립 문제_미장착/오장착   
4               W  2016     20160526         ZZ5         B11  조립 문제_미장착/오장착   

  trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
0         KV11          CK     J5          TT        82305   
1         KV11          YP     A9          TT        84211   
2         KV11          YP     A9          TT        83912   
3         KV11          YP     A9          TT        83120   
4         KV11          YP     A9          TT        83860   

                                        clm_prtl_sbc 대상 선정 구분 책임구 원인  \
0   엔진 룸 다량 연료 누유 발생 점검 엔진 룸 다량 연료 누유 점검 점검 내용 엔진...     X  N   N  N   
1                              리어 도어 안티 치핑 필름 장착 불량      X  N   N  N   
2                    커튼 홀더 장착 불량 조치 내용 커튼 홀더 장착 후 함      X  N   N  N   
3                및 조치 시 웨더 스트림 스트림 무 주름 발생 들뜸 교환 조치      X  N   N  N   
4                           몰딩 내부 고무 조립 불량 상태 몰딩 조립      X  N   N  N   

  target_pred  
0           X  
1           X  
2           X  
3           X  
4           X

## X, y 정의

In [45]:
from sklearn.preprocessing import LabelEncoder
y_encod = df['대상 선정']
classle = LabelEncoder()
y_encod = classle.fit_transform(df['대상 선정'].values) # y값 인코딩.
print(np.unique(y_encod))

[0 1]


In [48]:
# ro column을 list형태로 ro_text 변수에 할당.
ro_text = list(df['clm_prtl_sbc'])
sentiments = list(y_encod) # y값.

In [49]:
ro_text[0]

' 엔진 룸 다량 연료 누유 발생 점검 엔진 룸 다량 연료 누유 점검 점검 내용 엔진 룸 프런트 저압 라인 연료 공급 파이프 중 고압 펌프 연결 저압 라인 퀵 커넥터 터 오 체결 랭킹 중 다량 료 엔진 룸 내부 고압 연료 펌프 하단 바닥 누유 발생 함 고압 펌프 저압 라인 퀵 커넥터 터 오 체결 연료 펌프 구동 중 이탈 다량 연료 엔진 룸및 다닥 누유 사항 임 조치 내용 고압 펌측 퀵 턴텍터 체결 및 엔진 룸 가솔린 오염 부위 세척 시동 후 누유 확인 테스트 요망 사항 해당 저압 파이프 이탈 시 파이프 하단 부위 익 죠스 트 매니폴드 이 어서 가솔린 연료 발화 인하 회재 발생 수 있음 중요 부위 조립 철저 관리 필요 사항 임 '

In [50]:
sentiments[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [51]:
sentences = []
for text in ro_text:
    sentences.append(text.split())

In [12]:
sentences[:3]

[['엔진',
  '룸',
  '다량',
  '연료',
  '누유',
  '발생',
  '점검',
  '엔진',
  '룸',
  '다량',
  '연료',
  '누유',
  '점검',
  '점검',
  '내용',
  '엔진',
  '룸',
  '프런트',
  '저압',
  '라인',
  '연료',
  '공급',
  '파이프',
  '중',
  '고압',
  '펌프',
  '연결',
  '저압',
  '라인',
  '퀵',
  '커넥터',
  '터',
  '오',
  '체결',
  '랭킹',
  '중',
  '다량',
  '료',
  '엔진',
  '룸',
  '내부',
  '고압',
  '연료',
  '펌프',
  '하단',
  '바닥',
  '누유',
  '발생',
  '함',
  '고압',
  '펌프',
  '저압',
  '라인',
  '퀵',
  '커넥터',
  '터',
  '오',
  '체결',
  '연료',
  '펌프',
  '구동',
  '중',
  '이탈',
  '다량',
  '연료',
  '엔진',
  '룸및',
  '다닥',
  '누유',
  '사항',
  '임',
  '조치',
  '내용',
  '고압',
  '펌측',
  '퀵',
  '턴텍터',
  '체결',
  '및',
  '엔진',
  '룸',
  '가솔린',
  '오염',
  '부위',
  '세척',
  '시동',
  '후',
  '누유',
  '확인',
  '테스트',
  '요망',
  '사항',
  '해당',
  '저압',
  '파이프',
  '이탈',
  '시',
  '파이프',
  '하단',
  '부위',
  '익',
  '죠스',
  '트',
  '매니폴드',
  '이',
  '어서',
  '가솔린',
  '연료',
  '발화',
  '인하',
  '회재',
  '발생',
  '수',
  '있음',
  '중요',
  '부위',
  '조립',
  '철저',
  '관리',
  '필요',
  '사항',
  '임'],
 ['리어', '도어', '안티', '치핑', '필름', '장착',

## Word2Vec

In [52]:
# Word2Vec의 파라미터 값들 정의.
num_features = 300    
min_word_count = 40   
num_workers = 4       
context = 10          
downsampling = 1e-3 

In [53]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
   level=logging.INFO)

In [54]:
# word2vec으로 단어 학습.
from gensim.models import word2vec

model = word2vec.Word2Vec(sentences, workers=num_workers, \
           size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

2019-06-13 08:53:19,755 : INFO : collecting all words and their counts
2019-06-13 08:53:19,770 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-06-13 08:53:19,833 : INFO : PROGRESS: at sentence #10000, processed 271541 words, keeping 6764 word types
2019-06-13 08:53:19,848 : INFO : collected 6999 word types from a corpus of 287824 raw words and 10629 sentences
2019-06-13 08:53:19,848 : INFO : Loading a fresh vocabulary
2019-06-13 08:53:19,848 : INFO : effective_min_count=40 retains 711 unique words (10% of original 6999, drops 6288)
2019-06-13 08:53:19,848 : INFO : effective_min_count=40 leaves 258099 word corpus (89% of original 287824, drops 29725)
2019-06-13 08:53:19,864 : INFO : deleting the raw counts dictionary of 6999 items
2019-06-13 08:53:19,879 : INFO : sample=0.001 downsamples 66 most-common words
2019-06-13 08:53:19,879 : INFO : downsampling leaves estimated 162685 word corpus (63.0% of prior 258099)
2019-06-13 08:53:19,879 : INFO : estimated 

In [72]:
# Word2Vec의 feature vector를 얻는 함수.
def get_features(words, model, num_features):
    feature_vector = np.zeros((num_features),dtype=np.int32)

    num_words = 0
    index2word_set = set(model.wv.index2word)

    for w in words:
        if w in index2word_set:
            num_words += 1
            feature_vector = np.add(feature_vector, model[w])

    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [73]:
def get_dataset(ro_text, model, num_features):
    dataset = list()

    for s in ro_text:
        dataset.append(get_features(s, model, num_features))

    reviewFeatureVecs = np.stack(dataset)
    
    return reviewFeatureVecs

In [74]:
test_data_vecs = get_dataset(sentences, model, num_features)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


In [75]:
test_data_vecs

array([[ 0.06421514,  0.1547429 , -0.53828334, ...,  0.15341716,
         0.00547186, -0.01052636],
       [ 0.37493928,  0.03417365,  0.03393194, ...,  0.30295917,
        -0.09062699, -0.3262472 ],
       [ 0.16040172,  0.05555406, -0.15768493, ...,  0.1613936 ,
        -0.07427444, -0.36472196],
       ...,
       [ 0.10000647,  0.07007776, -0.36147868, ..., -0.00777582,
         0.05214262, -0.12038033],
       [ 0.20782854,  0.06757923, -0.39776468, ...,  0.08670742,
        -0.02379193,  0.0189068 ],
       [ 0.19166743,  0.08742672, -0.67528577, ...,  0.15582723,
         0.09334539, -0.20804567]])

In [76]:
test_data_vecs.shape

(10629, 300)

# 1. RNN

### Train, Test data Split - 1안 (RNN)

In [78]:
X = test_data_vecs
y = np.array(sentiments)

In [79]:
X.shape

(10629, 300)

In [80]:
y.shape

(10629,)

In [81]:
print(X)
print(y)

[[ 0.06421514  0.1547429  -0.53828334 ...  0.15341716  0.00547186
  -0.01052636]
 [ 0.37493928  0.03417365  0.03393194 ...  0.30295917 -0.09062699
  -0.3262472 ]
 [ 0.16040172  0.05555406 -0.15768493 ...  0.1613936  -0.07427444
  -0.36472196]
 ...
 [ 0.10000647  0.07007776 -0.36147868 ... -0.00777582  0.05214262
  -0.12038033]
 [ 0.20782854  0.06757923 -0.39776468 ...  0.08670742 -0.02379193
   0.0189068 ]
 [ 0.19166743  0.08742672 -0.67528577 ...  0.15582723  0.09334539
  -0.20804567]]
[1 1 1 ... 1 1 1]


In [82]:
maxlen = 200
training_samples = 9566
test_samples = 10000

In [83]:
indices = np.arange(X.shape[0])
indices2 = np.arange(y.shape[0])
np.random.shuffle(indices)
np.random.shuffle(indices2)
data_x = X[indices]
data_y = y[indices2]

In [84]:
x_train = data_x[:training_samples]
y_train = data_y[:training_samples]
x_test = data_x[training_samples: training_samples + test_samples]
y_test = data_y[training_samples: training_samples + test_samples]

In [85]:
print(x_train)

[[ 0.20293277  0.08107711 -0.19445415 ...  0.06065319  0.00571576
  -0.25373279]
 [ 0.40849139 -0.02745696 -0.12531067 ...  0.16080982 -0.14802721
   0.01838877]
 [ 0.14562843 -0.02444032 -0.13652006 ...  0.13616818 -0.0600492
   0.01113844]
 ...
 [ 0.32816947  0.02752999 -0.19612185 ...  0.26003121  0.17039152
  -0.22029365]
 [ 0.18156911  0.01170794  0.0806845  ...  0.37984242 -0.26456607
   0.24811899]
 [ 0.16989909 -0.06932613 -0.75598685 ... -0.1962143   0.28633578
  -0.37436415]]


In [86]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(9566, 300)
(9566,)
(1063, 300)
(1063,)


In [87]:
print(x_train)
print(y_train)
print(x_test)
print(y_test)

[[ 0.20293277  0.08107711 -0.19445415 ...  0.06065319  0.00571576
  -0.25373279]
 [ 0.40849139 -0.02745696 -0.12531067 ...  0.16080982 -0.14802721
   0.01838877]
 [ 0.14562843 -0.02444032 -0.13652006 ...  0.13616818 -0.0600492
   0.01113844]
 ...
 [ 0.32816947  0.02752999 -0.19612185 ...  0.26003121  0.17039152
  -0.22029365]
 [ 0.18156911  0.01170794  0.0806845  ...  0.37984242 -0.26456607
   0.24811899]
 [ 0.16989909 -0.06932613 -0.75598685 ... -0.1962143   0.28633578
  -0.37436415]]
[1 0 0 ... 0 1 0]
[[-0.04538094  0.03362155 -0.08611066 ...  0.13104036  0.0558818
  -0.18613788]
 [ 0.06440381  0.05126026 -0.17832757 ...  0.08271408  0.18337925
  -0.21339495]
 [ 0.21222296  0.04511424 -0.16672488 ...  0.15494559 -0.00233803
  -0.26823318]
 ...
 [-0.0056267   0.02834805 -0.11617211 ...  0.30496323  0.03319822
   0.04629631]
 [ 0.11275595 -0.06094635 -0.13182817 ...  0.23528528  0.09616404
  -0.05842536]
 [ 0.17850055 -0.1023608  -0.50502895 ... -0.03997291  0.14892501
  -0.13397491]]


In [107]:
from keras.models import Sequential
from keras.layers import Embedding,Dense,Flatten,SimpleRNN

max_features = 10000
maxlen = 300
embedding_dim = 300

model = Sequential()
model.add(Embedding(max_features,embedding_dim,input_length=maxlen))
# model.add(SimpleRNN(32))
model.add(Dense(32,activation='relu'))
model.add(Dense(3,activation='sigmoid'))

# model.add(Embedding(max_features,embedding_dim,input_length=maxlen))
# model.add(Flatten())
# model.add(Dense(32, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 300, 300)          3000000   
_________________________________________________________________
dense_21 (Dense)             (None, 300, 32)           9632      
_________________________________________________________________
dense_22 (Dense)             (None, 300, 3)            99        
Total params: 3,009,731
Trainable params: 3,009,731
Non-trainable params: 0
_________________________________________________________________


In [109]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
history = model.fit(x_train,y_train,epochs=10,batch_size=32,validation_data=(x_test,y_test))

ValueError: Error when checking target: expected dense_22 to have 3 dimensions, but got array with shape (9566, 1)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
print(history.history['acc'])
print(history.history['val_acc'])
print(history.history['loss'])
print(history.history['val_loss'])

In [ ]:
print(round(np.mean(history.history['acc'])*100,2))
print(round(np.mean(history.history['val_acc'])*100,2))

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

In [ ]:
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.figure()

In [ ]:
# model.load_weights('pre_trained_model.h5')
model.evaluate(x_test, y_test)

# 2. Random Forest

### Train, Test data Split - 2안 (RandomForest)

In [115]:
from sklearn.model_selection import train_test_split
import numpy as np

X = test_data_vecs
y = np.array(sentiments)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [116]:
from sklearn.model_selection import train_test_split
import numpy as np
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [117]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(9566, 300)
(1063, 300)
(9566,)
(1063,)


## RandomForestClassifier로 학습

In [118]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤포레스트 분류기를 사용
forest = RandomForestClassifier(
    n_estimators = 100, n_jobs = -1, random_state=42)
forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [119]:
%time forest = forest.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

## 모델평가

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=5, shuffle=True, random_state=0)

scoring = 'accuracy'
%time score = cross_val_score(forest, X_train, y_train, cv=k_fold, n_jobs=-1, scoring=scoring)
score

In [ ]:
round(np.mean(score)*100,2)

## 예측 결과

In [ ]:
from sklearn.metrics import accuracy_score
# 테스트 데이터를 넣고 예측합니다.
y_pred = forest.predict(X_test)
y_pred[:3]

In [ ]:
print(accuracy_score(y_test, y_pred))